# Movie Sentiment Analysis with Keras

In [0]:

SAMPLE_SIZE=50000
# SAMPLE_SIZE=1000
# TRAIN_TEST_RATIO=0.75
TRAIN_TEST_RATIO=0.50

BATCH_SIZE=64
EPOCHS_NB=3
vocab_size = 10000
# vocab_size = 1000

In [72]:
# uncomment these for Google collab, will have already been installed in local environment 
# if 'pip install -r requirements.txt' has been run
!pip install nltk wget
!pip install --upgrade gensim

import numpy as np
import os
import os.path

from pdb import set_trace
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
import nltk


import glob
from gensim.models import Word2Vec

import time

Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.7.1)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
# MacOSX: See https://www.mkyong.com/mac/wget-on-mac-os-x/ for wget
if not os.path.isfile('./glove.6B.300d.txt'):
    if not os.path.isfile('./glove.6B.zip'):
      !wget http://nlp.stanford.edu/data/glove.6B.zip 

    if not os.path.isfile('./glove.6B.300d.txt'):  
      !unzip glove.6B.zip 
      

In [74]:
import wget
import tarfile

# By checking if the directory exists first, we allow people to delete the tarfile without the notebook re-downloading it
if os.path.isdir('aclImdb'):
    print("Dataset directory exists, taking no action")
else:    
    if not os.path.isfile('aclImdb_v1.tar.gz'):
        print("Downloading dataset")
        #!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
        wget.download('http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz')
    else:
        print("Dataset already downloaded")
    
    print("Unpacking dataset")
    #!tar -xf aclImdb_v1.tar.gz 
    tar = tarfile.open("aclImdb_v1.tar.gz")
    tar.extractall()
    tar.close()
    print("Dataset unpacked in aclImdb")
      

import re

# load doc into memory
# regex to clean markup elements 
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r', encoding='utf8')
    # read all text
    text = re.sub('<[^>]*>', ' ', file.read())
    #text = file.read()
    # close the file
    file.close()
    return text


Dataset directory exists, taking no action


In [0]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# notebook configuration
TRAINING_SET_SIZE=int(SAMPLE_SIZE * (1 - TRAIN_TEST_RATIO))
VALIDATION_SET_SIZE=int(SAMPLE_SIZE * TRAIN_TEST_RATIO)

dataframes={}
for type in ["train","test"]:
    if type=="train":
        SLICE = int(TRAINING_SET_SIZE / 2)
    elif type=="test":
        SLICE = int(VALIDATION_SET_SIZE / 2)

    positive_file_list = glob.glob(os.path.join('aclImdb/'+type+'/pos', "*.txt"))
    positive_sample_file_list = positive_file_list[:SLICE]
    df_positive = pd.DataFrame({'reviews':[load_doc(x) for x in positive_sample_file_list], 'sentiment': np.ones(SLICE)})
    
    negative_file_list = glob.glob(os.path.join('aclImdb/'+type+'/neg', "*.txt"))
    negative_sample_file_list = negative_file_list[:SLICE]
    df_negative = pd.DataFrame({'reviews':[load_doc(x) for x in negative_sample_file_list], 'sentiment': np.zeros(SLICE)})

    dataframes[type]=pd.concat([df_positive,df_negative])
    dataframes[type]=shuffle(dataframes[type])

df=pd.concat([dataframes["train"],dataframes["test"]])
    
X_train=dataframes["train"]['reviews']
y_train=dataframes["train"]['sentiment']

X_test=dataframes["test"]['reviews']
y_test=dataframes["test"]['sentiment']

### Logic to compute DAN model

In [0]:
#ML STUDY GROUP
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np

class PreProcessor:
    def __init__(self,REVIEWS,REVIEWS_VAL,LABELS,LABELS_VAL,WE_FILE):
        self.reviews = REVIEWS
        self.reviews_val = REVIEWS_VAL
        self.labels = LABELS
        self.labels_val = LABELS_VAL
        self.we_file = WE_FILE

    def tokenize(self):
#         set_trace()
        print(self.reviews[0])

        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(self.reviews)

        self.sequences = tokenizer.texts_to_sequences(self.reviews)
        self.sequences_val = tokenizer.texts_to_sequences(self.reviews_val)

        self.word_index = tokenizer.word_index
        print("Found %s unique tokens" %(len(self.word_index)))

    def make_data(self):
        self.MAX_SEQUENCE_LENGTH = max([len(self.sequences[i]) for i in range(len(self.sequences))])
        print("self.MAX_SEQUENCE_LENGTH: {}".format(self.MAX_SEQUENCE_LENGTH))

        review = pad_sequences(self.sequences,maxlen=self.MAX_SEQUENCE_LENGTH)
        review_val = pad_sequences(self.sequences_val,maxlen=self.MAX_SEQUENCE_LENGTH)
        
        labels = to_categorical(self.labels)
        labels_val = to_categorical(self.labels_val)

        print("Shape of data tensor: " +str(review.shape))
        print("Shape of label tensor: " +str(labels.shape))

        return review, review_val, labels, labels_val
        
    def get_word_embedding_matrix(self,EMBEDDING_DIM=100):
        embeddings_index = {}

        if self.we_file == "rand":
            return None

        f = open(self.we_file)

        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
        f.close()

        print('Found %s word vectors.' % len(embeddings_index))

        self.embedding_matrix = np.zeros((len(self.word_index)+1, EMBEDDING_DIM))

        for word, i in self.word_index.items():
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                # words not found in embedding index will be all-zeros.
                self.embedding_matrix[i] = embedding_vector

        return self.embedding_matrix

from keras.layers import Layer
from keras import backend as K
import tensorflow as tf

class AverageWords(Layer):
    def __init__(self):
        super(AverageWords,self).__init__()
        self.supports_masking = True

    def call(self, x, mask=None):
        axis = K.ndim(x) - 2
        if mask is not None:
            summed = K.sum(x, axis=axis)
            n_words = K.expand_dims(K.sum(K.cast(mask, 'float32'), axis=axis), axis)
            return summed / n_words
        else:
            return K.mean(x, axis=axis)

    def compute_mask(self, inputs, mask=None):
        return None

    def compute_output_shape(self, input_shape):
        dimensions = list(input_shape)
        n_dimensions = len(input_shape)
        del dimensions[n_dimensions - 2]
        return tuple(dimensions)

class WordDropout(Layer):
    def __init__(self, rate):
        super(WordDropout,self).__init__()
        self.rate = min(1., max(0., rate))
        self.supports_masking = True

    def call(self, inputs, training=None):
        if 0. < self.rate < 1.0:
            def dropped_inputs():
                input_shape = K.shape(inputs)
                batch_size = input_shape[0]
                n_time_steps = input_shape[1]
                mask = tf.random_uniform((batch_size, n_time_steps, 1)) >= self.rate
                w_drop = K.cast(mask, 'float32') * inputs
                return w_drop
            return K.in_train_phase(dropped_inputs, inputs, training=training)
        return inputs

    def get_config(self):
        config = {'rate': self.rate}
        base_config = super().get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [0]:
import argparse
import numpy as np

from keras.layers import Embedding, Dense, Input, BatchNormalization, Activation, Dropout
from keras.models import Sequential
from keras.optimizers import Adagrad, Adam
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping

from pdb import set_trace

embedding_dim = 300
num_hidden_layers = 3
num_hidden_units = 300
num_epochs = 100
batch_size = 512
dropout_rate = 0.2
word_dropout_rate = 0.3
activation = 'relu'

args = {}
args['We']='./glove.6B.300d.txt'
args['Wels']='' ### rand or ''
args['model']='dan'  ### nbow OR dan
args['wd']='y'

reviews=X_train.values
reviews_val=X_test.values
labels=y_train.values
labels_val=y_test.values

In [0]:
# reviews_val.shape

In [79]:
pp = PreProcessor(reviews,reviews_val,labels,labels_val,args['We'])
pp.tokenize()

reviews,reviews_val,labels,labels_val = pp.make_data()

embedding_matrix = pp.get_word_embedding_matrix(embedding_dim)


I'm stunt, I must admit I never saw a movie with such good story and none stop high special effect martial art fighting scene. If you like the fantastic genre, like me, you will certainly be more than satisfied! All character have very cool power and the special effect are near perfection, in one word, flawless! I will listen to this movie a lot in the next years.
Found 88576 unique tokens
self.MAX_SEQUENCE_LENGTH: 2473
Shape of data tensor: (25000, 2473)
Shape of label tensor: (25000, 2)
Found 400000 word vectors.


In [0]:
model = Sequential()

if args['Wels'] == "rand":
    model.add(Embedding(len(pp.word_index) + 1,embedding_dim,input_length=pp.MAX_SEQUENCE_LENGTH,trainable=False))
else:
    model.add(Embedding(len(pp.word_index)+1,embedding_dim,weights=[embedding_matrix],input_length=pp.MAX_SEQUENCE_LENGTH,trainable=False))

if args['wd'] == 'y':
    model.add(WordDropout(word_dropout_rate))
model.add(AverageWords())


In [81]:
print('reviews.shape: ' + str(reviews.shape))
print('reviews_val.shape: ' + str(reviews_val.shape))
print('labels.shape: ' + str(labels.shape))
print('labels_val.shape: ' + str(labels_val.shape))
labels.shape[0]

reviews.shape: (25000, 2473)
reviews_val.shape: (25000, 2473)
labels.shape: (25000, 2)
labels_val.shape: (25000, 2)


25000

In [82]:
if args['model'] == 'dan':
    for i in range(num_hidden_layers):
        model.add(Dense(num_hidden_units))
        model.add(BatchNormalization())
        model.add(Activation(activation))
        model.add(Dropout(dropout_rate))

model.add(Dense(labels.shape[1]))
model.add(BatchNormalization())
model.add(Dropout(dropout_rate))
model.add(Activation('softmax'))

adam = Adam()
model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy','categorical_accuracy'])

model.summary()

model_checkpoint = ModelCheckpoint('best.weights', monitor='val_loss', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

callbacks = [model_checkpoint, early_stopping]

history = model.fit(reviews,labels,batch_size=batch_size,epochs=num_epochs,\
          validation_data=(reviews_val,labels_val), callbacks=callbacks)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 2473, 300)         26573100  
_________________________________________________________________
word_dropout_3 (WordDropout) (None, 2473, 300)         0         
_________________________________________________________________
average_words_3 (AverageWord (None, 300)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 300)               90300     
_________________________________________________________________
batch_normalization_13 (Batc (None, 300)               1200      
_________________________________________________________________
activation_13 (Activation)   (None, 300)               0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 300)               0         
__________

In [83]:
df = pd.DataFrame(history.history)
df=df[df['val_acc']==df.val_acc.max()]
df.reset_index(inplace=True)
df["title"]=["Keras DAN"]
df["sample_size"]=[SAMPLE_SIZE]
df["nb_epochs"]=[df.iloc[0]["index"]+1]
df.drop(labels="index",axis=1,inplace=True)
print(df)
df.to_csv(path_or_buf=df.iloc[0].title+".csv")

       acc  categorical_accuracy     loss  val_acc  val_categorical_accuracy  \
0  0.81544               0.81544  0.41265  0.83408                   0.83408   

   val_loss      title  sample_size  nb_epochs  
0  0.371968  Keras DAN        50000          9  
